In [1]:
import pandas as pd
from datetime import datetime

In [2]:
digital_music = pd.read_json('/Users/aakankshajoshi/Downloads/Digital_Music_5.json', lines=True)

In [3]:
digital_music_filtered = digital_music[['asin', 'reviewerID', 'reviewTime', 'overall']]
digital_music_filtered['reviewTime'] = pd.to_datetime(digital_music_filtered['reviewTime'])
digital_music_filtered['timeDiff'] = (pd.to_datetime('2014-07-23') - digital_music_filtered['reviewTime']).dt.days
print(digital_music_filtered.head())

/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


         asin      reviewerID reviewTime  overall  timeDiff
0  5555991584  A3EBHHCZO6V2A4 2006-09-12        5      2871
1  5555991584   AZPWAXJG9OJXV 2001-06-03        5      4798
2  5555991584  A38IRL0X2T4DPF 2003-07-14        5      4027
3  5555991584  A22IK3I6U76GX0 2000-05-03        5      5194
4  5555991584  A1AISPOIIHTHXX 2008-01-17        4      2379


/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [4]:
from tqdm import tqdm

rating = {}

for i in tqdm(range(len(digital_music_filtered.reviewerID))):
    if digital_music_filtered.reviewerID[i] in rating: 
        rating[digital_music_filtered.reviewerID[i]] = rating[digital_music_filtered.reviewerID[i]] + digital_music_filtered.overall[i]
    else:
        rating[digital_music_filtered.reviewerID[i]] = digital_music_filtered.overall[i]
        
rating_df = pd.DataFrame([rating.keys(), rating.values()])
rating_df = rating_df.T
rating_df.rename(columns={0: "reviewerID", 1: "totalRating"}, inplace=True)

joined_data = digital_music_filtered.join(rating_df.set_index('reviewerID'), on='reviewerID')
joined_data['totalRating'] = pd.to_numeric(joined_data['totalRating'])
joined_data['ratingMetric'] = pd.to_numeric(joined_data['overall']/joined_data['totalRating'])
print(joined_data.head())

100%|██████████| 64706/64706 [00:03<00:00, 19713.76it/s]


         asin      reviewerID reviewTime  overall  timeDiff  totalRating  \
0  5555991584  A3EBHHCZO6V2A4 2006-09-12        5      2871          325   
1  5555991584   AZPWAXJG9OJXV 2001-06-03        5      4798           80   
2  5555991584  A38IRL0X2T4DPF 2003-07-14        5      4027           51   
3  5555991584  A22IK3I6U76GX0 2000-05-03        5      5194           26   
4  5555991584  A1AISPOIIHTHXX 2008-01-17        4      2379           36   

   ratingMetric  
0      0.015385  
1      0.062500  
2      0.098039  
3      0.192308  
4      0.111111  


In [6]:
from pyspark.sql import SQLContext
from pyspark import SparkContext

sc = SparkContext()
sqlContext = SQLContext(sc)

In [11]:
digital_music_metadata = sqlContext.read.json("/Users/aakankshajoshi/Downloads/meta_Digital_Music.json")
digital_music_filtered_metadata = digital_music_metadata[['asin', 'brand', 'description', 'price', 'title']]
digital_music_filtered_metadata.show()

+----------+-----+--------------------+-----+--------------------+
|      asin|brand|         description|price|               title|
+----------+-----+--------------------+-----+--------------------+
|5555991584| null|                null| 9.49|     Memory of Trees|
|6308051551| null|NEW Combo BLUWAVS...| 8.91|Don't Drink His B...|
|7901622466| null|                null|11.33|             On Fire|
|B0000000ZW|     |                    |23.64|      Changing Faces|
|B00000016W| null|                null| 9.49|          Pet Sounds|
|B00000016T| null|                null|14.63|                Cars|
|      null| null|                null| null|                null|
|B00000017R| null|                null| 8.99|          Jazz Samba|
|B0000001BO| null|                null| 8.99|          Solar Heat|
|B0000001O0| null|Soulero by Kenny ...|14.98|             Soulero|
|B0000001PS| null|                null| 7.92|        Browne Sugar|
|B0000001Q8| null|                null| 8.99|Live at the Blue 

In [16]:
joined_data_spark = sqlContext.createDataFrame(joined_data)
all_music = joined_data_spark.join(digital_music_filtered_metadata, ["asin"])
all_music.show()

+----------+--------------+-------------------+-------+--------+-----------+--------------------+-----+--------------------+-----+----------+
|      asin|    reviewerID|         reviewTime|overall|timeDiff|totalRating|        ratingMetric|brand|         description|price|     title|
+----------+--------------+-------------------+-------+--------+-----------+--------------------+-----+--------------------+-----+----------+
|B000000OOE| AF349JR6MDNWA|2000-11-20 00:00:00|      5|    4993|         63| 0.07936507936507936| null|        Beck ~ Loser| 4.99|     Loser|
|B000000OOE|A2INX8B6VXUVW6|2005-07-08 00:00:00|      4|    3302|        184|0.021739130434782608| null|        Beck ~ Loser| 4.99|     Loser|
|B000000OOE|A2HR4MLTTXC23A|2002-04-12 00:00:00|      4|    4485|         76| 0.05263157894736842| null|        Beck ~ Loser| 4.99|     Loser|
|B000000OOE|A14GK0E64J0WAS|2003-08-09 00:00:00|      5|    4001|        767| 0.00651890482398957| null|        Beck ~ Loser| 4.99|     Loser|
|B0000